In [1]:
#This code computes medians of errors and uses bootstrapping to calculate errors of medians

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.gridspec as gridspec
import ast
import sys
sys.path.append('machine-scientist/')
sys.path.append('machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.stats import bootstrap

In [3]:
#Read error dataframe for all simulations
function='tanh'
errors=pd.read_csv('../data/'+ 'errors_' + str(function) + '.csv',index_col=0)
display(errors)

,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test,n,r
0,0.0,0.038346,0.031069,0.001119,0.009678,0.048355,0.034498,0.001325,0.011692,0,0
1,0.0,0.011996,0.106369,0.002785,0.332193,0.015287,0.112211,0.003340,0.497563,1,0
2,0.0,0.045330,0.568604,0.001012,0.058189,0.119820,0.574555,0.001299,0.072254,2,0
3,0.0,0.017690,0.168191,0.000309,0.041747,0.043636,0.169878,0.000399,0.048745,3,0
4,0.0,0.015367,0.066641,0.000170,0.002949,0.019958,0.070403,0.000248,0.004091,4,0
...,...,...,...,...,...,...,...,...,...,...,...
325,0.2,0.059918,0.041152,0.025700,0.111698,0.077847,0.049771,0.039826,0.118180,5,2
326,0.2,0.138111,0.465183,0.047928,0.275630,0.168871,0.470017,0.074429,0.277328,6,2
327,0.2,0.091140,0.182207,0.095380,0.024869,0.117414,0.183915,0.110881,0.027818,7,2
328,0.2,0.111839,0.209496,0.070767,1.238419,0.135749,0.217007,0.076842,1.503048,8,2


In [4]:
#Calculate medians over 30 simulations and drop 'maes'
columns_errors=['rmse_nn_train','rmse_nn_test','rmse_mdl_train','rmse_mdl_test']
medians=errors.groupby(['sigma'],as_index=False)[columns_errors].median()
display(medians)

,sigma,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test
0,0.00,0.017356,0.088850,0.000520,0.039035
1,0.02,0.021286,0.079044,0.011320,0.131624
2,0.04,0.025385,0.100456,0.023240,0.038125
3,0.06,0.032712,0.164878,0.027520,0.103755
4,0.08,0.049341,0.091179,0.042324,0.110910
5,0.10,0.054660,0.155441,0.050440,0.095145
6,0.12,0.080489,0.181612,0.055367,0.131610
7,0.14,0.078299,0.102278,0.076024,0.194697
8,0.16,0.080551,0.237049,0.050001,0.181378
9,0.18,0.117082,0.137691,0.097960,0.341428


In [5]:
sigmas=[0.0, 0.02, 0.04,0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]
median_errors={}

for sigma in sigmas:
    for i in columns_errors: 

        #----------------------------
    
        #----------------------------
        
        error=(list(medians[i]),)
        bs_iterations=20;
        bs_medians=[];bs_distributions=[]
        
        for j in range(bs_iterations):
            bs=bootstrap(error, np.median, n_resamples=30, confidence_level=0.9, random_state=None)
            bs_dist=bs.bootstrap_distribution; bs_distributions.append(bs_dist)
            median_bs=np.median(bs_dist)
            bs_medians.append(median_bs)

        try:
            median_errors['error_' +i].append(np.var(bs_medians))
        except KeyError:
            median_errors['error_' +i]=[np.var(bs_medians)]

#Convert dict of median errors to dataframe                                              
median_errors_df=pd.DataFrame.from_dict(median_errors)
median_errors_df['sigma']=sigmas
display(median_errors_df)

#Join error dataframe to medians dataframe
medians=medians.join(median_errors_df.set_index('sigma'), on='sigma')
medians.to_csv('../data/medians_' + str(function) + '.csv')
display(medians)

,error_rmse_nn_train,error_rmse_nn_test,error_rmse_mdl_train,error_rmse_mdl_test,sigma
0,0.000008,0.000060,5.709648e-06,0.000024,0.00
1,0.000035,0.000023,1.336827e-06,0.000069,0.02
2,0.000029,0.000184,3.447710e-06,0.000005,0.04
3,0.000031,0.000183,3.433986e-06,0.000057,0.06
4,0.000019,0.000051,3.951862e-06,0.000020,0.08
5,0.000054,0.000094,3.420021e-06,0.000039,0.10
6,0.000061,0.000049,3.433986e-06,0.000024,0.12
7,0.000027,0.000039,1.975692e-08,0.000020,0.14
8,0.000002,0.000247,3.933926e-06,0.000028,0.16
9,0.000075,0.000096,5.760936e-06,0.000024,0.18


,sigma,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test,error_rmse_nn_train,error_rmse_nn_test,error_rmse_mdl_train,error_rmse_mdl_test
0,0.00,0.017356,0.088850,0.000520,0.039035,0.000008,0.000060,5.709648e-06,0.000024
1,0.02,0.021286,0.079044,0.011320,0.131624,0.000035,0.000023,1.336827e-06,0.000069
2,0.04,0.025385,0.100456,0.023240,0.038125,0.000029,0.000184,3.447710e-06,0.000005
3,0.06,0.032712,0.164878,0.027520,0.103755,0.000031,0.000183,3.433986e-06,0.000057
4,0.08,0.049341,0.091179,0.042324,0.110910,0.000019,0.000051,3.951862e-06,0.000020
5,0.10,0.054660,0.155441,0.050440,0.095145,0.000054,0.000094,3.420021e-06,0.000039
6,0.12,0.080489,0.181612,0.055367,0.131610,0.000061,0.000049,3.433986e-06,0.000024
7,0.14,0.078299,0.102278,0.076024,0.194697,0.000027,0.000039,1.975692e-08,0.000020
8,0.16,0.080551,0.237049,0.050001,0.181378,0.000002,0.000247,3.933926e-06,0.000028
9,0.18,0.117082,0.137691,0.097960,0.341428,0.000075,0.000096,5.760936e-06,0.000024
